# NN Learning MNIST in Google Colab

In [3]:
import os
from pathlib import Path
import json
import pandas as pd

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models

import matplotlib.pyplot as plt
import numpy as np

## Load Data

In [2]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0

# Onehot-Encoding
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

## Set Hyperparameters

In [14]:
session_name = "test1"

layer_counts = [1, 2]
layer_neuron_counts = [16, 32]
layer_activation_types = ["sigmoid", "relu"]

epochs = 5

## Create Model

In [9]:
def create_model(layer_count, layer_neurons, layer_activations):
    model = models.Sequential()
    
    model.add(layers.Flatten(input_shape=(28,28)))
    
    for n in range(layer_count):
        model.add(layers.Dense(layer_neurons[n], activation=layer_activations[n]))
    
    model.add(layers.Dense(10, activation="softmax"))
    
    model.compile(
            loss="categorical_crossentropy",
        metrics=["accuracy"],
        optimizer="adam"
    )
    
    return model
        

In [15]:
def create_model_dir(model_name):
    model_dir = "../../models/tf_colab_mnist/"+ session_name + "/" + model_name + "/"

    saved_dir = model_dir + "saved/"
    checkpoint_dir = model_dir + "cp/"
    history_dir = model_dir + "history/"

    checkpoint_path = checkpoint_dir + "cp-{epoch:02d}.ckpt"

    Path(saved_dir).mkdir(parents=True, exist_ok=True)
    Path(checkpoint_dir).mkdir(parents=True, exist_ok=True)
    Path(history_dir).mkdir(parents=True, exist_ok=True)
    
    return saved_dir, checkpoint_dir, history_dir

In [17]:
for layer_count in layer_counts:
    for layer_neuron_count in layer_neuron_counts:
        for layer_activation_type in layer_activation_types

1
2


In [16]:
model = create_model(2, [8, 10], ["relu", "sigmoid"])
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_2 (Flatten)         (None, 784)               0         
                                                                 
 dense_4 (Dense)             (None, 8)                 6280      
                                                                 
 dense_5 (Dense)             (None, 10)                90        
                                                                 
 dense_6 (Dense)             (None, 10)                110       
                                                                 
Total params: 6,480
Trainable params: 6,480
Non-trainable params: 0
_________________________________________________________________
